In [ ]:
!pip install fasttext tqdm --quiet

In [ ]:
from tqdm import tqdm
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
input_path='/kaggle/Input/'
output_path='/kaggle/working/'

## Preprocess

### Load the dataset

In [ ]:
df= pd.read_parquet(input_path+'Dataset/collected/acl-publication-info.74k.parquet')
df

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,International Journal of Computational Linguis...,"Chen, Keh-Jiann and\nYou, Jia-Ming",A Study on Word Similarity using Context Vecto...,37--58,None,None,None,None,None,None
1,L02-1310,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Mihalcea, Rada F.",Bootstrapping Large Sense Tagged Corpora,None,None,None,None,None,None,None
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Jamison, Emily and\nGurevych, Iryna","Headerless, Quoteless, but not Hopeless? Using...",327--335,None,None,None,None,None,None
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Aswani, Niraj and\nGaizauskas, Robert",Aligning Words in {E}nglish-{H}indi Parallel C...,115--118,None,None,None,None,None,None
4,L02-1309,,,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Suyaga, Fumiaki and\nTakezawa, Toshiyuki and...",Proposal of a very-large-corpus acquisition me...,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73280,P99-1002,This paper describes recent progress and the a...,This paper describes recent progress and the a...,715160,ab17a01f142124744c6ae425f8a23011366ec3ee,11,https://aclanthology.org/P99-1002,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Furui, Sadaoki",Automatic Speech Recognition and Its Applicati...,11--20,10.3115/1034678.1034680,None,None,None,None,None
73281,P00-1009,We present an LFG-DOP parser which uses fragme...,We present an LFG-DOP parser which uses fragme...,1356246,ad005b3fd0c867667118482227e31d9378229751,12,https://aclanthology.org/P00-1009,Association for Computational Linguistics,Hong Kong,2000,...,Proceedings of the 38th Annual Meeting of the ...,"Bod, Rens",An Improved Parser for Data-Oriented Lexical-F...,61--68,10.3115/1075218.1075227,None,None,None,None,None
73282,P99-1056,The processes through which readers evoke ment...,The processes through which readers evoke ment...,7277828,924cf7a4836ebfc20ee094c30e61b949be049fb6,14,https://aclanthology.org/P99-1056,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Lange, Marielle and\nContent, Alain",The grapho-phonological system of written {F}r...,436--442,10.3115/1034678.1034745,None,None,None,None,None
73283,P99-1051,This paper examines the extent to which verb d...,This paper examines the extent to which verb d...,1829043,6b1f6f28ee36de69e8afac39461ee1158cd4d49a,92,https://aclanthology.org/P99-1051,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Lapata, Maria",Acquiring Le

In [ ]:
df.columns

Index(['acl_id', 'abstract', 'full_text', 'corpus_paper_id', 'pdf_hash',
       'numcitedby', 'url', 'publisher', 'address', 'year', 'month',
       'booktitle', 'author', 'title', 'pages', 'doi', 'number', 'volume',
       'journal', 'editor', 'isbn'],
      dtype='object')

### Remove the invalid data

In [ ]:
data=df
remove_list=[]
for i in df.index:
    # there are some empty full_text examples also some with really limited text
    # the reason set to 2000 words (~10000 bytes), partialy reference: https://quantifyinghealth.com/length-of-a-research-paper/ and the quatile number above
    if len(data.full_text[i])<10000:
        remove_list.append(i)
data=data.drop(remove_list)
data.reset_index(inplace=True,drop = True)
data

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,International Journal of Computational Linguis...,"Chen, Keh-Jiann and\nYou, Jia-Ming",A Study on Word Similarity using Context Vecto...,37--58,None,None,None,None,None,None
1,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Jamison, Emily and\nGurevych, Iryna","Headerless, Quoteless, but not Hopeless? Using...",327--335,None,None,None,None,None,None
2,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Aswani, Niraj and\nGaizauskas, Robert",Aligning Words in {E}nglish-{H}indi Parallel C...,115--118,None,None,None,None,None,None
3,R13-1044,The paper 1 presents a rule-based approach to ...,The paper 1 presents a rule-based approach to ...,2491460,c0f1047fe0f95c367184d494e78bb07b11ee3608,2,https://aclanthology.org/R13-1044,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"K{\k{e}}dzia, Pawe{\l} and\nMaziarz, Marek",Recognizing semantic relations within {P}olish...,342--349,None,None,None,None,None,None
4,W05-0818,"In this paper we describe LIHLA, a lexical ali...","In this paper we describe LIHLA, a lexical ali...",15322146,ff3f05120d24e5dac2879f25402993bc6355f780,5,https://aclanthology.org/W05-0818,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Caseli, Helena M. and\nNunes, Maria G. V. an...",{LIHLA}: Shared Task System Description,111--114,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61169,P99-1002,This paper describes recent progress and the a...,This paper describes recent progress and the a...,715160,ab17a01f142124744c6ae425f8a23011366ec3ee,11,https://aclanthology.org/P99-1002,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Furui, Sadaoki",Automatic Speech Recognition and Its Applicati...,11--20,10.3115/1034678.1034680,None,None,None,None,None
61170,P00-1009,We present an LFG-DOP parser which uses fragme...,We present an LFG-DOP parser which uses fragme...,1356246,ad005b3fd0c867667118482227e31d9378229751,12,https://aclanthology.org/P00-1009,Association for Computational Linguistics,Hong Kong,2000,...,Proceedings of the 38th Annual Meeting of the ...,"Bod, Rens",An Improved Parser for Data-Oriented Lexical-F...,61--68,10.3115/1075218.1075227,None,None,None,None,None
61171,P99-1056,The processes through which readers evoke ment...,The processes through which readers evoke ment...,7277828,924cf7a4836ebfc20ee094c30e61b949be049fb6,14,https://aclanthology.org/P99-1056,Association for Computational Linguistics,"College Park, Maryland, USA",1999,...,Proceedings of the 37th Annual Meeting of the ...,"Lange, Marielle and\nContent, Alain",The grapho-phonological system of written {F}r...,436--442,10.3115/1034678.1034745,None,None,None,None,None
61172,P99-1051,This paper examines the extent to which verb d...,This paper examines the extent to which verb d...,1829043,6b1f6f28ee36de69e8afac39461ee1158cd4d49a,92,https://aclanthology.org/P99-1051,Association fo

In [ ]:
data=data[['url', 'title', 'full_text','year', 'month']]
data

,url,title,full_text,year,month
0,https://aclanthology.org/O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,2002,August
1,https://aclanthology.org/R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,2013,September
2,https://aclanthology.org/W05-0819,Aligning Words in {E}nglish-{H}indi Parallel C...,"In this paper, we describe a word alignment al...",2005,June
3,https://aclanthology.org/R13-1044,Recognizing semantic relations within {P}olish...,The paper 1 presents a rule-based approach to ...,2013,September
4,https://aclanthology.org/W05-0818,{LIHLA}: Shared Task System Description,"In this paper we describe LIHLA, a lexical ali...",2005,June
...,...,...,...,...,...
61169,https://aclanthology.org/P99-1002,Automatic Speech Recognition and Its Applicati...,This paper describes recent progress and the a...,1999,June
61170,https://aclanthology.org/P00-1009,An Improved Parser for Data-Oriented Lexical-F...,We present an LFG-DOP parser which uses fragme...,2000,October
61171,https://aclanthology.org/P99-1056,The grapho-phonological system of written {F}r...,The processes through which readers evoke ment...,1999,June
61172,https://aclanthology.org/P99-1051,Acquiring Lexical Generalizations from Corpora...,This paper examines the extent to which verb d...,1999,June


### Filter out the papers not written in English

In [ ]:
import fasttext
model_path = path+'/Models/lid.176.bin'  # path to the pre-trained FastText language identification model
model = fasttext.load_model(model_path)

In [ ]:
lan_label=[]
for _, item in tqdm(data.iterrows(), total=len(data)):
# for item in data.full_text:
  result = model.predict(item.full_text)
  lan_label.append(result[0][0][9:])

100%|██████████| 61174/61174 [05:03<00:00, 201.81it/s]


In [ ]:
remove_list=[]
for i,label in enumerate(lan_label):
  if label!='en':
    remove_list.append(i)
data_en=data
data_en=data_en.drop(remove_list)
data_en.reset_index(inplace=True,drop = True)
data_en

,url,title,full_text,year,month
0,https://aclanthology.org/O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,2002,August
1,https://aclanthology.org/R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,2013,September
2,https://aclanthology.org/W05-0819,Aligning Words in {E}nglish-{H}indi Parallel C...,"In this paper, we describe a word alignment al...",2005,June
3,https://aclanthology.org/R13-1044,Recognizing semantic relations within {P}olish...,The paper 1 presents a rule-based approach to ...,2013,September
4,https://aclanthology.org/W05-0818,{LIHLA}: Shared Task System Description,"In this paper we describe LIHLA, a lexical ali...",2005,June
...,...,...,...,...,...
59198,https://aclanthology.org/P99-1002,Automatic Speech Recognition and Its Applicati...,This paper describes recent progress and the a...,1999,June
59199,https://aclanthology.org/P00-1009,An Improved Parser for Data-Oriented Lexical-F...,We present an LFG-DOP parser which uses fragme...,2000,October
59200,https://aclanthology.org/P99-1056,The grapho-phonological system of written {F}r...,The processes through which readers evoke ment...,1999,June
59201,https://aclanthology.org/P99-1051,Acquiring Lexical Generalizations from Corpora...,This paper examines the extent to which verb d...,1999,June


### Filter out special symbols in text

In [ ]:
pattern = r'acknowledgment|acknowledgments'
def truncate(text):
    match_text = re.search(pattern, text, flags=re.IGNORECASE)
    if match_text:
        return text[:match_text.start()]
    else:
        return text

In [ ]:
# Apply the extraction function to the text column
data_en['text']=data_en['full_text'].apply(truncate)
data_en

,url,title,full_text,year,month,text
0,https://aclanthology.org/O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,2002,August,There is a need to measure word similarity whe...
1,https://aclanthology.org/R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,2013,September,Thread disentanglement is the task of separati...
2,https://aclanthology.org/W05-0819,Aligning Words in {E}nglish-{H}indi Parallel C...,"In this paper, we describe a word alignment al...",2005,June,"In this paper, we describe a word alignment al..."
3,https://aclanthology.org/R13-1044,Recognizing semantic relations within {P}olish...,The paper 1 presents a rule-based approach to ...,2013,September,The paper 1 presents a rule-based approach to ...
4,https://aclanthology.org/W05-0818,{LIHLA}: Shared Task System Description,"In this paper we describe LIHLA, a lexical ali...",2005,June,"In this paper we describe LIHLA, a lexical ali..."
...,...,...,...,...,...,...
59198,https://aclanthology.org/P99-1002,Automatic Speech Recognition and Its Applicati...,This paper describes recent progress and the a...,1999,June,This paper describes recent progress and the a...
59199,https://aclanthology.org/P00-1009,An Improved Parser for Data-Oriented Lexical-F...,We present an LFG-DOP parser which uses fragme...,2000,October,We present an LFG-DOP parser which uses fragme...
59200,https://aclanthology.org/P99-1056,The grapho-phonological system of written {F}r...,The processes through which readers evoke ment...,1999,June,The processes through which readers evoke ment...
59201,https://aclanthology.org/P99-1051,Acquiring Lexical Generalizations from Corpora...,This paper examines the extent to which verb d...,1999,June,This paper examines the extent to which verb d...


In [ ]:
def filter_content(text):
    allowed_symbols = ['"', '`', "'","/" ,"\\",'(', ')', '[', ']', ".", ':', ",", "?", "!",'-']
    # website_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    # pattern = r"[^\w\s{}]+|(?:{})".format(re.escape(''.join(allowed_symbols)), website_pattern)
    pattern = r"[^\w\s{}]".format(re.escape(''.join(allowed_symbols)))
    filtered_text = re.sub(pattern, '', text)
    return filtered_text

In [ ]:
data_en['text']=pd.Series(dtype='object')
for i in data_en.index:
    text=data_en.title[i]+'\n'+data_en.full_text[i]
    data_en['text'][i]=filter_content(text)
data_en

,url,title,full_text,year,month,text
0,https://aclanthology.org/O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,2002,August,A Study on Word Similarity using Context Vecto...
1,https://aclanthology.org/R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,2013,September,"Headerless, Quoteless, but not Hopeless? Using..."
2,https://aclanthology.org/W05-0819,Aligning Words in {E}nglish-{H}indi Parallel C...,"In this paper, we describe a word alignment al...",2005,June,Aligning Words in English-Hindi Parallel Corpo...
3,https://aclanthology.org/R13-1044,Recognizing semantic relations within {P}olish...,The paper 1 presents a rule-based approach to ...,2013,September,Recognizing semantic relations within Polish n...
4,https://aclanthology.org/W05-0818,{LIHLA}: Shared Task System Description,"In this paper we describe LIHLA, a lexical ali...",2005,June,LIHLA: Shared Task System Description\nIn this...
...,...,...,...,...,...,...
59198,https://aclanthology.org/P99-1002,Automatic Speech Recognition and Its Applicati...,This paper describes recent progress and the a...,1999,June,Automatic Speech Recognition and Its Applicati...
59199,https://aclanthology.org/P00-1009,An Improved Parser for Data-Oriented Lexical-F...,We present an LFG-DOP parser which uses fragme...,2000,October,An Improved Parser for Data-Oriented Lexical-F...
59200,https://aclanthology.org/P99-1056,The grapho-phonological system of written {F}r...,The processes through which readers evoke ment...,1999,June,The grapho-phonological system of written Fren...
59201,https://aclanthology.org/P99-1051,Acquiring Lexical Generalizations from Corpora...,This paper examines the extent to which verb d...,1999,June,Acquiring Lexical Generalizations from Corpora...


In [ ]:
data_tar= data_en.drop(columns=['title','full_text'])
data_tar

,url,year,month,text
0,https://aclanthology.org/O02-2002,2002,August,A Study on Word Similarity using Context Vecto...
1,https://aclanthology.org/R13-1042,2013,September,"Headerless, Quoteless, but not Hopeless? Using..."
2,https://aclanthology.org/W05-0819,2005,June,Aligning Words in English-Hindi Parallel Corpo...
3,https://aclanthology.org/R13-1044,2013,September,Recognizing semantic relations within Polish n...
4,https://aclanthology.org/W05-0818,2005,June,LIHLA: Shared Task System Description\nIn this...
...,...,...,...,...
59198,https://aclanthology.org/P99-1002,1999,June,Automatic Speech Recognition and Its Applicati...
59199,https://aclanthology.org/P00-1009,2000,October,An Improved Parser for Data-Oriented Lexical-F...
59200,https://aclanthology.org/P99-1056,1999,June,The grapho-phonological system of written Fren...
59201,https://aclanthology.org/P99-1051,1999,June,Acquiring Lexical Generalizations from Corpora...


In [ ]:
len(data_tar)

59203

### Convert text into sentences

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
# 20000 examples take  ~50min
# due to the limited RAM, seperate the dataset into 3 files
data_tar1=data_tar
# data_tar1=data_tar[0:20000]
# data_tar1=data_tar[20000:40000]
# data_tar1=data_tar[40000:len(data_tar)]
data_tar1['sentences']=pd.Series(dtype='object')
for i, item in tqdm(data_tar.iterrows(), total=len(data_tar1)):
  sentences=sent_tokenize(item.text, language='english')
  data_tar1['sentences'][i]=sentences
data_tar1

100%|██████████| 59203/59203 [06:52<00:00, 143.40it/s]


,url,year,month,text,sentences
0,https://aclanthology.org/O02-2002,2002,August,A Study on Word Similarity using Context Vecto...,[A Study on Word Similarity using Context Vect...
1,https://aclanthology.org/R13-1042,2013,September,"Headerless, Quoteless, but not Hopeless? Using...","[Headerless, Quoteless, but not Hopeless?, Usi..."
2,https://aclanthology.org/W05-0819,2005,June,Aligning Words in English-Hindi Parallel Corpo...,[Aligning Words in English-Hindi Parallel Corp...
3,https://aclanthology.org/R13-1044,2013,September,Recognizing semantic relations within Polish n...,[Recognizing semantic relations within Polish ...
4,https://aclanthology.org/W05-0818,2005,June,LIHLA: Shared Task System Description\nIn this...,[LIHLA: Shared Task System Description\nIn thi...
...,...,...,...,...,...
59198,https://aclanthology.org/P99-1002,1999,June,Automatic Speech Recognition and Its Applicati...,[Automatic Speech Recognition and Its Applicat...
59199,https://aclanthology.org/P00-1009,2000,October,An Improved Parser for Data-Oriented Lexical-F...,[An Improved Parser for Data-Oriented Lexical-...
59200,https://aclanthology.org/P99-1056,1999,June,The grapho-phonological system of written Fren...,[The grapho-phonological system of written Fre...
59201,https://aclanthology.org/P99-1051,1999,June,Acquiring Lexical Generalizations from Corpora...,[Acquiring Lexical Generalizations from Corpor...


In [ ]:
# change file name every time
data_tar1.to_parquet(output_path+"/Dataset/preprocessed/target_sentences.parquet")

### Statistical overview

In [ ]:
target_sentences= pd.read_parquet(input_path+"/Dataset/preprocessed/target_sentences.parquet")
target_sentences

,url,year,month,text,sentences
0,https://aclanthology.org/O02-2002,2002,August,A Study on Word Similarity using Context Vecto...,[A Study on Word Similarity using Context Vect...
1,https://aclanthology.org/R13-1042,2013,September,"Headerless, Quoteless, but not Hopeless? Using...","[Headerless, Quoteless, but not Hopeless?, Usi..."
2,https://aclanthology.org/W05-0819,2005,June,Aligning Words in English-Hindi Parallel Corpo...,[Aligning Words in English-Hindi Parallel Corp...
3,https://aclanthology.org/R13-1044,2013,September,Recognizing semantic relations within Polish n...,[Recognizing semantic relations within Polish ...
4,https://aclanthology.org/W05-0818,2005,June,LIHLA: Shared Task System Description\nIn this...,[LIHLA: Shared Task System Description\nIn thi...
...,...,...,...,...,...
59198,https://aclanthology.org/P99-1002,1999,June,Automatic Speech Recognition and Its Applicati...,[Automatic Speech Recognition and Its Applicat...
59199,https://aclanthology.org/P00-1009,2000,October,An Improved Parser for Data-Oriented Lexical-F...,[An Improved Parser for Data-Oriented Lexical-...
59200,https://aclanthology.org/P99-1056,1999,June,The grapho-phonological system of written Fren...,[The grapho-phonological system of written Fre...
59201,https://aclanthology.org/P99-1051,1999,June,Acquiring Lexical Generalizations from Corpora...,[Acquiring Lexical Generalizations from Corpor...


Sentences

In [ ]:
df_sen_len=target_sentences['sentences'].apply(len)
df_sen_len

0        161
1        203
2        114
3        141
4         58
        ... 
59198    158
59199    123
59200    113
59201    197
59202    147
Name: sentences, Length: 59203, dtype: int64

In [ ]:
df_sen_len.describe()

count    59203.000000
mean       176.651690
std         79.854688
min          1.000000
25%        123.000000
50%        169.000000
75%        214.000000
max       4490.000000
Name: sentence, dtype: float64

Tokens

In [ ]:
# takes about 30 mins to run
from nltk.tokenize import word_tokenize
from tqdm import tqdm
df_tok=pd.DataFrame()
df_tok.index=target_sentences.index
df_tok['len']=pd.Series(dtype='object')
for i, item in tqdm(target_sentences.iterrows(), total=len(target_sentences)):
  len_tokens=0
  for sentence in item.sentences:
    len_tokens+=len(word_tokenize(sentence, language='english'))
  # tokens = [[token for token in word_tokenize(sentence, language='english')] for sentence in item]
  print(len_tokens)
  df_tok['len'][i]=len_tokens
df_tok

In [ ]:
df_tok.describe()

count     59203.000000
mean       4577.295374
std        2456.530602
min        1086.000000
25%        3125.000000
50%        4393.000000
75%        5496.500000
max      125078.000000
Name: tokens, dtype: float64